# Pré-processamento

## Importações iniciais

In [48]:
import pandas as pd 
import numpy as np


from sklearn.preprocessing import OrdinalEncoder 
from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier

from category_encoders import CatBoostEncoder


In [64]:
colunas_dummies = ['Churn', 
                   'customer.Partner', 
                   'customer.Dependents',
                   'customer.gender',
                   'phone.PhoneService', 
                   'account.PaperlessBilling']

colunas_multiclasses = ['internet.InternetService', 
                        'account.Contract',
                        'account.PaymentMethod']

colunas_dummies_mais_Um = ['phone.MultipleLines', 
                        'internet.OnlineSecurity', 
                        'internet.OnlineBackup',
                        'internet.DeviceProtection', 
                        'internet.TechSupport', 
                        'internet.StreamingTV',
                        'internet.StreamingMovies']

In [50]:
data_bruto = pd.read_csv("JsonConvertido.csv")

In [51]:
data_bruto.head()

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


### Transformação das variáveis dummies 

o padrão costuma ser
- (yes, no) -> (1,0)
- (male, female) -> (1,0)

In [52]:
data_bruto[colunas_dummies].head()

,Churn,customer.Partner,customer.Dependents,customer.gender,phone.PhoneService,account.PaperlessBilling
0,No,Yes,Yes,Female,Yes,Yes
1,No,No,No,Male,Yes,No
2,Yes,No,No,Male,Yes,Yes
3,Yes,Yes,No,Male,Yes,Yes
4,Yes,Yes,No,Female,Yes,Yes


In [53]:
ordinal_encoder = OrdinalEncoder()

In [54]:
data_bruto[colunas_dummies] = ordinal_encoder.fit_transform(data_bruto[colunas_dummies])

In [57]:
data_bruto[colunas_dummies].head()

,Churn,customer.Partner,customer.Dependents,customer.gender,phone.PhoneService,account.PaperlessBilling
0,0.0,1.0,1.0,0.0,1.0,1.0
1,0.0,0.0,0.0,1.0,1.0,0.0
2,1.0,0.0,0.0,1.0,1.0,1.0
3,1.0,1.0,0.0,1.0,1.0,1.0
4,1.0,1.0,0.0,0.0,1.0,1.0


### Separação em treino e teste 

Isso é feio para permitir que o modelo treine, nos dados de treino e que possamos ver sua eficiência no teste e também evitar vazamento de dados

In [58]:
id = data_bruto["customerID"]
id.sample(2)

1471    2108-XWMPY
4678    6407-UTSLV
Name: customerID, dtype: object

In [59]:
X = data_bruto.drop(["Churn", "customerID"], axis=1)
y = data_bruto["Churn"]

In [60]:
y.head()

0    0.0
1    0.0
2    1.0
3    1.0
4    1.0
Name: Churn, dtype: float64

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Transformação das colunas com Yes, no e mais um fator

In [69]:
for col in data_bruto[colunas_dummies_mais_Um].columns:
    print(f'{col} : {data_bruto[col].unique()}')

phone.MultipleLines : ['No' 'Yes' 'No phone service']
internet.OnlineSecurity : ['No' 'Yes' 'No internet service']
internet.OnlineBackup : ['Yes' 'No' 'No internet service']
internet.DeviceProtection : ['No' 'Yes' 'No internet service']
internet.TechSupport : ['Yes' 'No' 'No internet service']
internet.StreamingTV : ['Yes' 'No' 'No internet service']
internet.StreamingMovies : ['No' 'Yes' 'No internet service']


In [ ]:
def transforma(row):
    #row[colunas_dummies_mais_Um] = [lambda x: 1 if row]